In [2]:
%load_ext autoreload
%autoreload 2

In [320]:
import mplleaflet
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from osm2nx import read_osm, haversine
from graph import (
    states_to_state_avenue_name, subset_graph_by_edge_name, keep_oneway_edges_only, create_connected_components,
    identify_kinked_nodes, create_unkinked_connected_components, nodewise_distance_connected_components,
    calculate_component_overlap, calculate_redundant_components, contract_edges, create_deduped_state_road_graph, 
    create_contracted_edge_graph,shortest_paths_between_components, find_minimum_weight_edges_to_connect_components,
    create_rpp_edgelist
    )

from postman_problems.tests.utils import create_mock_csv_from_dataframe
from postman_problems.solver import rpp, cpp
from postman_problems.stats import calculate_postman_solution_stats

## Load OSM to NetworkxX

In [4]:
%%time

g = read_osm('../../50states/district-of-columbia-latest.osm')  

<class 'networkx.classes.digraph.DiGraph'>
CPU times: user 55.7 s, sys: 2.54 s, total: 58.2 s
Wall time: 1min 1s


## Make Graph w State Avenues only

#### Generate state avenue names

In [5]:
STATE_STREET_NAMES = [
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado",
    "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
    "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
    "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
    "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
    "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
    "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
    "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"
]

In [6]:
candidate_state_avenue_names = states_to_state_avenue_name(STATE_STREET_NAMES)

#### Create graph w state avenues only

In [7]:
g_st = subset_graph_by_edge_name(g, candidate_state_avenue_names)

In [8]:
# Add state edge attribute from full streetname (with avenue/drive and quandrant)
for e in g_st.edges(data=True):
    e[2]['state'] = e[2]['name'].rsplit(' ', 2)[0]

## Remove redundant state avenues

#### create state avenue graph w oneway edges only

In [523]:
g_st1 = keep_oneway_edges_only(g_st)

#### create simple connected components

(these are oneway state avenues only)

In [524]:
comps = create_connected_components(g_st1)

#### remove kinked nodes

In [534]:
comps_unkinked = create_unkinked_connected_components(comps, 60)

# comps in dict form for easy lookup
comps_dict = {comp.graph['id']:comp for comp in comps_unkinked} 

#### matching connected components

In [535]:
# caclulate nodewise distances between each node in comp with closest node in each candidate
comp_matches = nodewise_distance_connected_components(comps_unkinked)

# calculate overlap between components
comp_overlap = calculate_component_overlap(comp_matches, thresh_distance=75)

# identify redundant and non-redundant components
remove_comp_ids, keep_comp_ids = calculate_redundant_components(comp_overlap, thresh_pct=0.75)

#### viz redundant component solution

In [536]:
fig, ax = plt.subplots()

for i, road in enumerate(remove_comp_ids):
    for comp_id in remove_comp_ids[road]:
        comp = comps_dict[comp_id]
        posc = {k: (float(comp.node[k]['lon']), float(comp.node[k]['lat'])) for k in comp.nodes()}
        width = 7.0
        nx.draw_networkx_edges(comp, posc, width=width, edge_color='red', arrows=False)
        #nx.draw_networkx_nodes(comp, posc, node_size=40, nodelist=comp.nodes(), node_color='red', edge_color='k', alpha=1, with_labels=True)

for i, road in enumerate(keep_comp_ids):
    for comp_id in keep_comp_ids[road]:
        comp = comps_dict[comp_id]
        posc = {k: (float(comp.node[k]['lon']), float(comp.node[k]['lat'])) for k in comp.nodes()}
        width = 3.0
        nx.draw_networkx_edges(comp, posc, width=width, edge_color='black', arrows=False)
        #nx.draw_networkx_nodes(comp, posc, node_size=40, nodelist=comp.nodes(), node_color='red', edge_color='k', alpha=1, with_labels=True)        

pos_st = {k: (float(g_st.node[k]['lon']), float(g_st.node[k]['lat'])) for k in g_st.nodes()}    
nx.draw_networkx_edges(g_st, pos_st, width=1.0, edge_color='blue', alpha=0.7, arrows=False)

mplleaflet.show(fig=ax.figure)

## Create single connected component

In [248]:
# create a single graph with deduped state roads
g_st_nd = create_deduped_state_road_graph(g_st, comps_dict, remove_comp_ids)

In [112]:
# Create graph with contracted edges only
g_st_contracted = create_contracted_edge_graph(g_st_nd, 'length')

In [113]:
# create dataframe with shortest paths between each component
dfsp = shortest_paths_between_components(g_st_contracted)

In [114]:
g_ud = g.to_undirected()  # necessary for min weight func below

# min weight edges that create a single connected component
connector_edges = find_minimum_weight_edges_to_connect_components(dfsp, g_ud, 'length', 10)

In [522]:
connector_edges[0]

('3378364625',
 '2071884010',
 {'distance': 2.423094275667694, 'path': ['3378364625', '2071884010']})

In [115]:
# adding connector edges to create one single connected component
for e in connector_edges:
    g_st_contracted.add_edge(e[0], e[1], distance=e[2]['distance'], path=e[2]['path'], required=1, connector=True)
    
print(sum([e[2]['distance'] for e in g_st_contracted.edges(data=True)])/1600)

118.55355310056581


### Viz: single connected component

#### make graph with granular edges connecting components

In [140]:
g1component = g_st_contracted.copy()
for e in g_st_contracted.edges(data=True):
    if 'path' in e[2]:
        granular_type = 'connector' if 'connector' in e[2] else 'state'
        
        # add granular connector edges to graph 
        for pair in list(zip(e[2]['path'][:-1], e[2]['path'][1:])):
            g1component.add_edge(pair[0], pair[1], granular='True', granular_type=granular_type)
            
        # add granular connector nodes to graph
        for n in e[2]['path']:
            g1component.add_node(n, lat=g.node[n]['lat'], lon=g.node[n]['lon'])

#### Make granular connected viz

In [254]:
fig, ax = plt.subplots()

g1component_subconnector = g1component.copy()
g1component_st = g1component.copy()

for e in g1component.edges(data=True):
    if ('granular_type' not in e[2]) or (e[2]['granular_type'] != 'connector'):
        g1component_subconnector.remove_edge(e[0], e[1])

for e in g1component.edges(data=True):
    if ('granular_type' not in e[2]) or (e[2]['granular_type'] != 'state'):
        g1component_st.remove_edge(e[0], e[1])
    
pos = {k: (float(g1component_subconnector.node[k]['lon']), float(g1component_subconnector.node[k]['lat'])) for k in g1component_subconnector.nodes()}    
nx.draw_networkx_edges(g1component_subconnector, pos, width=5, edge_color='red', arrows=False)

pos_st = {k: (float(g1component_st.node[k]['lon']), float(g1component_st.node[k]['lat'])) for k in g1component_st.nodes()}    
nx.draw_networkx_edges(g1component_st, pos_st, width=3, edge_color='black', arrows=False)

mplleaflet.show(fig=ax.figure)

## Solve CPP

### Create CPP edgelist

In [145]:
tmp = []
for e in g_st_contracted.edges(data=True):
    tmpi = e[2].copy()  # so we don't mess w original graph
    tmpi['start_node'] = e[0]
    tmpi['end_node'] = e[1]
    tmp.append(tmpi)

# create dataframe w node1 and node2 in order
tmpdf = pd.DataFrame(tmp)   
tmpdf = tmpdf[['start_node', 'end_node'] + list(set(tmpdf.columns)-{'start_node', 'end_node'})]

In [347]:
tmpdf.head(3)

,start_node,end_node,required,path,connector,distance,comp,name
0,3010850857,3010850865,1,"[3010850857, 3010850864, 3010850865]",NaN,22.970120,90.0,Delaware Avenue Northeast
1,1243974093,49882304,1,"[1243974093, 49882304]",NaN,9.539226,10.0,New Hampshire Avenue Northwest
2,1243974093,49882294,1,"[49882294, 1243974201, 1243974200, 1243974199,...",True,31.335871,NaN,NaN


###  CPP solver

In [147]:
# create mockfilename
elfn = create_mock_csv_from_dataframe(tmpdf)

# solve
circuit_cpp, gcpp = cpp(elfn)

# circuit stats
calculate_postman_solution_stats(circuit_cpp)
print(sum([e[3]['distance'] for e in circuit_cpp])/1600)

236.92620192711135


## Solve RPP


`get_shortest_paths_distances` is the bottleneck applying djkstra path length on all possible combinations.  there are ~14k pairs to calculate shortest path for (4 per second) which would take almost one hour.

#### create edgelist for RPP

This adds optional edges whose distance is less than 1600m.  The corresponding granular path is added as an attribute, but not used for computation to improve efficiency of solver.

In [706]:
%%time
dfrpp = create_rpp_edgelist(g_st_contracted, g_ud, 'length', 3200)

CPU times: user 39min 59s, sys: 28.1 s, total: 40min 28s
Wall time: 41min 52s


In [707]:
pd.value_counts(dfrpp['required'])

0    14894
1      333
Name: required, dtype: int64

TODO: The path edge attribute in `circuit_rpp` is correct.  and comes from the `dfrpp`.  However it is in string form, rather than list... not sure why.  can fix with exec, but that's messy.

TODO: explore multigraph

In [708]:
%%time

# create mockfilename
elfn = create_mock_csv_from_dataframe(dfrpp)

# solve
circuit_rpp, grpp = rpp(elfn, verbose=True)

# circuit stats
print(sum([e[3]['distance'] for e in circuit_rpp])/1600)

INFO:postman_problems.solver.rpp:read edgelist
INFO:postman_problems.solver.rpp:create full and required graph
INFO:postman_problems.solver.rpp:getting odd node pairs
INFO:postman_problems.solver.rpp:get shortest paths between odd nodes
INFO:postman_problems.solver.rpp:Find min weight matching using blossom algorithm
INFO:postman_problems.solver.rpp:add the min weight matching edges to g
INFO:postman_problems.solver.rpp:get eulerian circuit route


153.8387683536511
CPU times: user 7min 31s, sys: 4.31 s, total: 7min 36s
Wall time: 7min 49s


In [709]:
# hack to convert 'path' from str back to list.  Caused by `create_mock_csv_from_dataframe`
for e in circuit_rpp:
    if type(e[3]['path']) == str:
        exec('e[3]["path"]=' + e[3]["path"])

In [710]:
calculate_postman_solution_stats(circuit_rpp)

OrderedDict([('distance_walked', 246142.02936584174),
             ('distance_doublebacked', 56456.34440493667),
             ('distance_walked_once', 189685.68496090506),
             ('distance_walked_optional', 51391.89182604927),
             ('distance_walked_required', 194750.13753979246),
             ('edges_walked', 423),
             ('edges_doublebacked', 90),
             ('edges_walked_once', 333),
             ('edges_walked_optional', 60),
             ('edges_walked_required', 363)])

Results Log:
- 203.5127938140708 miles excluding optional roads > 400
- 169.2348234543054 miles excluding optional roads > 800  
- 151.43473114349692 miles excluding optional roads > 1600  
- 144.1187638086149 miles excluding optional roads > 3200



### Viz RPP graph

#### create RPP granular graph

TODO: This shortest path calc should not be needed anymore... we should be able to grab this from `path` in circuit_rpp.

In [411]:
optional_edges = [[doc[0], doc[1]] for doc in circuit_rpp if doc[3]['required']==0]

In [410]:
# calc shortest path between optional nodes and add to g1component graph
for e in optional_edges:
    
    # add granular edges to g1component
    # TODO: shouldn't need this anymore since path is in edge attributes
    path = nx.dijkstra_path(g_ud, e[0], e[1], 'length')
    for pair in list(zip(path[:-1], path[1:])):
        g1component.add_edge(pair[0], pair[1], granular='True', granular_type='optional')
    
    # add granular connector nodes to graph
    for n in path:
        g1component.add_node(n, lat=g.node[n]['lat'], lon=g.node[n]['lon'])


In [573]:
circuit_rpp[1]

('3010850865',
 '3010850854',
 0,
 {'distance': 17.269866171653618,
  'distance_haversine': 16.393431366587933,
  'id': 383,
  'path': ['3010850865', '3010850866', '3010850854'],
  'required': 1})

#### create graphs and visualize

In [574]:
fig, ax = plt.subplots()

g1component_subconnector = g1component.copy()
g1component_st = g1component.copy()
g1component_opt = g1component.copy()

for e in g1component.edges(data=True):
    if e[2].get('granular_type') != 'connector':
        g1component_subconnector.remove_edge(e[0], e[1])

for e in g1component.edges(data=True):
    if e[2].get('granular_type') != 'state':
        g1component_st.remove_edge(e[0], e[1])

for e in g1component.edges(data=True):
    if e[2].get('granular_type') != 'optional':
        g1component_opt.remove_edge(e[0], e[1])

        
pos = {k: (float(g1component_subconnector.node[k]['lon']), float(g1component_subconnector.node[k]['lat'])) for k in g1component_subconnector.nodes()}    
nx.draw_networkx_edges(g1component_subconnector, pos, width=6, edge_color='blue', arrows=False)

pos_st = {k: (float(g1component_st.node[k]['lon']), float(g1component_st.node[k]['lat'])) for k in g1component_st.nodes()}    
nx.draw_networkx_edges(g1component_st, pos_st, width=4, edge_color='black', arrows=False)

pos_opt = {k: (float(g1component_opt.node[k]['lon']), float(g1component_opt.node[k]['lat'])) for k in g1component_opt.nodes()}    
nx.draw_networkx_edges(g1component_opt, pos_opt, width=2, edge_color='blue', arrows=False)

mplleaflet.show(fig=ax.figure)

### Viz circuit_rpp solution

In [720]:
## Create graph directly from rpp_circuit and original graph w lat/lon (g_ud)
color_seq = [None, 'black', 'red', 'purple']
grppviz = nx.Graph()
edges_cnt = Counter([tuple(sorted([e[0], e[1]])) for e in circuit_rpp])

for e in circuit_rpp:
    for n1, n2 in zip(e[3]['path'][:-1], e[3]['path'][1:]):
        if grppviz.has_edge(n1, n2):
            grppviz[n1][n2]['linewidth'] += 2
            grppviz[n1][n2]['cnt'] += 1
        else:                
            grppviz.add_edge(n1, n2, linewidth=2.5)
            grppviz[n1][n2]['color_st'] = 'black' if g_st.has_edge(n1, n2) else 'red'
            grppviz[n1][n2]['cnt'] = 1
            grppviz.add_node(n1, lat=g_ud.node[n1]['lat'], lon=g_ud.node[n1]['lon'])
            grppviz.add_node(n2, lat=g_ud.node[n2]['lat'], lon=g_ud.node[n2]['lon']) 

for e in grppviz.edges(data=True):
    e[2]['color_cnt'] = color_seq[e[2]['cnt']]


In [721]:
fig, ax = plt.subplots()

pos = {k: (float(grppviz.node[k]['lon']), float(grppviz.node[k]['lat'])) for k in grppviz.nodes()}    
e_width = [e[2]['linewidth'] for e in grppviz.edges(data=True)]
e_color = [e[2]['color_cnt'] for e in grppviz.edges(data=True)]
nx.draw_networkx_edges(grppviz, pos, width=e_width, edge_color=e_color, arrows=False, alpha=0.8)

mplleaflet.show(fig=ax.figure)


/Users/ajb/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
